In [1]:
import os
import sys
import datetime

import matlab
import pandas as pd
import numpy as np
import csv

os.environ["LD_LIBRARY_PATH"] = "/home/renan/MATLAB/R2021b/runtime/glnxa64/"

sys.path.append("./matlab/myBTD2/myBTD2/for_redistribution_files_only")
import myBTD2

sys.path.append("./matlab/myBTD/myBTD/for_redistribution_files_only")
import myBTD

from myPackages import myDehankelization, myHankelization

In [2]:
def get_dim_hankel(N):
    N = int(N)
    if N & 1:
        I = J = (N + 1) / 2
    else:
        I = N / 2
        J = N / 2 + 1
    return int(I), int(J)


def hankelization(recording):
    n_lead, n_rec = recording.shape
    I, J = get_dim_hankel(n_rec)
    T = np.zeros((I, J, n_lead))
    my_hankelization = myHankelization.initialize()
    for i in range(0, n_lead):
        vec = matlab.double(recording[i].tolist(), size=recording[i].shape)
        T[:, :, i] = my_hankelization.hankelization(vec)
    my_hankelization.terminate()
    T = matlab.double(T.tolist(), size=T.shape)
    return T


def run_btd(params):
    btd = myBTD.initialize()
    result, output = btd.myBTD(
        params["rank"],
        params["multirank"],
        params["max_iter"],
        params["tol_fun"],
        params["tol_x"],
        params["display"],
        params["recording_hankel"],
        nargout=2,
    )
    btd.terminate()
    return result, output


def run_btd2(params):
    btd = myBTD2.initialize()
    result, output = btd.myBTD2(
        params["rank"],
        params["lr"],
        params["max_iter"],
        params["tol_fun"],
        params["tol_x"],
        params["display"],
        params["recording_hankel"],
        nargout=2,
    )
    btd.terminate()
    return result, output

In [3]:
MULTIRANK = matlab.double([40, 40, 1], size=(1, 3))
LR = matlab.double([17], size=(1, 1))
MAX_ITER = matlab.double([1000], size=(1, 1))
TOLERANCE_FUN = matlab.double([1.0e-6], size=(1, 1))
TOLERANCE_X = matlab.double([1.0e-6], size=(1, 1))
DISPLAY = matlab.logical([False], size=(1, 1))
path = f"../workdata/shura/"

LL1_MODE = True

if not os.path.exists(f"{path}"):
    os.makedirs(path)

In [ ]:
rows = aioros_df.shape[0]

for rank in range(5, 8):
    print(f"RANK: {rank}")

    if not os.path.exists(f"{path}/rank_{rank}"):
        os.makedirs(f"{path}/rank_{rank}")

    rank_matlab = matlab.double([rank], size=(1, 1))

    for index, row in aioros_df.iterrows():
        recording = row["data"]
        recording_hankel = hankelization(recording)
        print(f"{index}/{rows}")

        params = {
            "rank": rank_matlab,
            "lr": LR,
            "multirank": MULTIRANK,
            "max_iter": MAX_ITER,
            "tol_fun": TOLERANCE_FUN,
            "tol_x": TOLERANCE_X,
            "display": DISPLAY,
            "recording_hankel": recording_hankel,
        }

        result, output = run_btd2(params) if LL1_MODE else run_btd(params)

        output["index"] = index
        output["rank"] = rank
        output["lr"] = LR[0][0]
        output["filename"] = row["filename"]
        output["db"] = row["db"]
        output["diagnostic"] = row["diag"]
        output["datetime"] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

        with open(f"{path}/rank_{rank}/{row.name}.npy", "wb") as file:
            np.save(file, result)

        file_exists = os.path.exists(f"{path}/results.csv")

        with open(f"{path}/results.csv", "a") as f:
            headers = output.keys()
            writer = csv.DictWriter(
                f, delimiter=",", lineterminator="\n", fieldnames=headers
            )
            if not file_exists:
                writer.writeheader()
            writer.writerow(output)